In [68]:
import math
import random
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import scipy.stats as stats
from scipy.stats import invgamma

In [69]:
# 一些全局变量，放在这里好设置
# 说白了就是可以在合理范围之内改动的参数

# SHM-Data的位置
SHM_Data_Locate = "D://SHM-Data/"

# 连续小波变换的输入宽度
# 对现在用的SHM-Data来说够了，改大了就是徒增计算量
tof_widths = 101

# CF/RP板材的大小(mm)
# 没啥用，哈哈哈
plant_size = [500, 500]

# 12组传感器的xy坐标
sensor_locate = np.array([[450, 470], [370, 470], [290, 470], [210, 470], [130, 470], [50, 470], [450, 30], [370, 30], [290, 30], [210, 30], [130, 30], [50, 30]])

# 28个缺陷的位置
damage_locate = np.array([
    [50, 415], [65, 415], [50, 400], [65, 400],
    [250, 427], [265, 427], [250,412], [265, 412],
    [180, 345], [195, 345], [180,330], [195, 345],
    [320, 275], [335, 275], [320, 260], [335, 275],
    [130, 195], [145, 195], [130, 180], [130, 180],
    [435, 205], [450, 205], [435, 190], [450, 190],
    [250, 90], [265, 90], [250, 75], [265, 75]
])

# 选用的baseline
SHM_Data_baseline_locate = '20180604T164628_baseline_1/pc_f100kHz.h5'

# MCMC总运行次数
# 越大越准，越大计算越慢
NT = 1000

# MCMC老化时间
# 只要你NT设置的足够大，老化时间是可以忽略的，可惜我懒得等计算
NB = 150

# 先验概率密度函数
# 正态分布的参数
[def_p_x, def_p_y] = [180, 345]
[def_p_x_scale, def_p_y_scale] = [10, 10]

# 计算取得波速Vg的大小
vg = 2000

# RAPID中控制椭圆有效分布区域大小的缩放系数
beta = 1.05

# 选用的数据组
select = np.array([[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11]])

In [70]:
# 使用连续小波变换获取ToF时间
# 输入catch信号和频道，输出ToF时间
def def_tof(def_tof_catch, def_tof_channel):
    print("def_tof", def_tof_catch, def_tof_channel)
    def_tof_data = def_tof_catch[()][def_tof_channel, :]
    def_tof_widths = np.arange(1, tof_widths)
    def_tof_data_cwt = signal.cwt(def_tof_data, signal.morlet2, def_tof_widths)
    def_tof_data_cwt = np.square(np.abs(def_tof_data_cwt))
    def_tof_max_x, def_tof_max_y = np.where(def_tof_data_cwt == np.max(def_tof_data_cwt))
    def_tof_select = def_tof_data_cwt[def_tof_max_x, :]
    def_tof_select.resize(def_tof_select.size)
    def_tof_peak = signal.find_peaks(def_tof_select)
    def_tof_tof = def_tof_peak[0][1] - def_tof_peak[0][0]
    return def_tof_tof

In [71]:
def def_get_array(def_get_locate):
    print("def_get_array", def_get_locate)
    def_get_file = h5py.File(SHM_Data_Locate + def_get_locate, 'r')
    def_get_catch = def_get_file['/pitchcatch/catch']
    def_get_array1 = def_get_catch[()]
    return def_get_array1

In [72]:
# 对所有catch信号获取ToF时间，返回选中的几组
def def_multi_tof(dmt_locate):
    print("def_multi_tof", dmt_locate)
    dmt_file = h5py.File(SHM_Data_Locate + dmt_locate, 'r')
    dmt_catch = dmt_file['/pitchcatch/catch']
    dmt_data_list = np.zeros((66,))
    for i in range(0, 66):
        dmt_data_list[i] = def_tof(dmt_catch, i)
    dmt_data_result = np.zeros((12, 12))
    dmt_data_point = 0
    for i in range(0, 12):
        for j in range(i + 1, 12):
            dmt_data_result[i][j] = dmt_data_list[dmt_data_point]
            dmt_data_result[j][i] = dmt_data_list[dmt_data_point]
            dmt_data_point += 1
    dmt_array_x = np.size(select, 0)
    dmt_data_select = np.zeros(dmt_array_x,)
    for i in range(dmt_array_x):
        dmt_data_select[i] = dmt_data_result[select[i][0]][select[i][1]]
    dmt_array = def_get_array(dmt_locate)
    return dmt_data_select, dmt_array

In [73]:
def def_rd(def_rd_select, def_rd_x, def_rd_y):
    print("def_rd", def_rd_select, def_rd_x, def_rd_y)
    def_rd_xi = sensor_locate[def_rd_select[0]][0]
    def_rd_yi = sensor_locate[def_rd_select[0]][1]
    def_rd_xj = sensor_locate[def_rd_select[1]][0]
    def_rd_yj = sensor_locate[def_rd_select[1]][1]
    def_dr_xy = (np.sqrt(np.square(def_rd_x - def_rd_xi) + np.square(def_rd_y - def_rd_yi)) + np.sqrt(np.square(def_rd_x - def_rd_xj) + np.square(def_rd_y - def_rd_yj))) / np.sqrt(np.square(def_rd_xi - def_rd_xj) + np.square(def_rd_yi - def_rd_yj))
    return def_dr_xy

In [74]:
def def_ri(def_ri_select, def_ri_x, def_ri_y):
    print("def_ri", def_ri_select, def_ri_x, def_ri_y)
    def_ri_ri = def_rd(def_ri_select, def_ri_x, def_ri_y)
    if def_ri_ri >= beta:
        return beta
    else:
        return def_ri_ri

In [75]:
def def_cxy(def_cxy_dmg, def_cxy_baseline):
    print("def_cxy", def_cxy_dmg, def_cxy_baseline)
    def_cxy_dmg_mean = def_cxy_dmg.mean()
    def_cxy_baseline_mean = def_cxy_baseline.mean()
    def_cxy_count = 0
    for i in range(def_cxy_dmg.size):
        def_cxy_count += (def_cxy_dmg[i] - def_cxy_dmg_mean) * (def_cxy_baseline[i] - def_cxy_baseline_mean)
    return def_cxy_count

In [76]:
def def_sigma(def_sigma_dmg, def_sigma_baseline):
    print("def_sigma", def_sigma_dmg, def_sigma_baseline)
    def_sigma_dmg_mean = def_sigma_dmg.mean()
    def_sigma_baseline_mean = def_sigma_baseline.mean()
    def_sigma_x = 0
    def_sigma_y = 0
    for i in range(def_sigma_dmg.size):
        def_sigma_x += np.square(def_sigma_dmg[i] - def_sigma_dmg_mean)
        def_sigma_y += np.square(def_sigma_baseline[i] - def_sigma_baseline_mean)
    return def_sigma_x * def_sigma_y

In [77]:
def def_aij(def_aij_dmg, def_aij_baseline):
    print("def_aij", def_aij_dmg, def_aij_baseline)
    rho = def_cxy(def_aij_dmg, def_aij_baseline) - def_sigma(def_aij_dmg, def_aij_baseline)
    return 1 - rho

In [78]:
def fuc_rapid_pdf2(fuc_rapid_pdf2_array, fuc_rapid_pdf2_num, fuc_rapid_pdf2_select):
    print("fuc_rapid_pdf2", fuc_rapid_pdf2_array, fuc_rapid_pdf2_num, fuc_rapid_pdf2_select)
    fuc_rapid_pdf2_pdf = np.zeros(plant_size)
    fuc_rapid_pdf2_dmg = fuc_rapid_pdf2_array[int(fuc_rapid_pdf2_num), :]
    fuc_rapid_pdf2_baseline = SHM_Data_baseline[int(fuc_rapid_pdf2_num), :]
    for i in range(np.size(fuc_rapid_pdf2_pdf, 0)):
        for j in range(np.size(fuc_rapid_pdf2_pdf, 1)):
            fuc_rapid_pdf2_pdf[i][j] = def_aij(fuc_rapid_pdf2_dmg, fuc_rapid_pdf2_baseline) * (beta - def_ri(fuc_rapid_pdf2_select, i, j)) / (beta - 1)
    return fuc_rapid_pdf2_pdf

In [79]:
def fuc_rapid_pdf(fuc_rapid_pdf_array):
    print("fuc_rapid_pdf", fuc_rapid_pdf_array)
    fuc_rapid_pdf_pdf = np.zeros(plant_size)
    fuc_p_array_count = 0
    fuc_p_array_select = np.zeros((12, 12))
    for i in range(0, 12):
        for j in range(i + 1, 12):
            fuc_p_array_select[i][j] = fuc_p_array_count
            fuc_p_array_select[j][i] = fuc_p_array_count
            fuc_p_array_count += 1
    for i in range(np.size(select, 0)):
        fuc_rapid_pdf_pdf += fuc_rapid_pdf2(fuc_rapid_pdf_array, fuc_p_array_select[select[i][0]][select[i][1]], select[i])
    fuc_rapid_pdf_pdf = fuc_rapid_pdf_pdf / np.size(select, 0)
    return fuc_rapid_pdf_pdf

In [80]:
# 先验概率密度函数之比
def fuc_p(fuc_p_pdf_pdf, fuc_p_theta1, fuc_p_theta2):
    print("fuc_p", fuc_p_pdf_pdf, fuc_p_theta1, fuc_p_theta2)
    fuc_p_x1 = fuc_p_theta1[0]
    fuc_p_y1 = fuc_p_theta1[1]
    fuc_p_x2 = fuc_p_theta2[0]
    fuc_p_y2 = fuc_p_theta2[1]
    fuc_p1 = fuc_p_pdf_pdf[fuc_p_x1][fuc_p_y1]
    fuc_p2 = fuc_p_pdf_pdf[fuc_p_x2][fuc_p_y2]
    fuc_pp = fuc_p1 / fuc_p2
    return min(1, fuc_pp)

In [81]:
# 计算理论与实际的误差
def fuc_q(fuc_q_theta, def_q_tof):
    print("fuc_q", fuc_q_theta, def_q_tof)
    xd = fuc_q_theta[0]
    yd = fuc_q_theta[1]
    count = 0
    def_q_size = np.size(select, 0)
    for i in range(def_q_size):
        count += np.square(def_q_tof[1] - np.sqrt(np.square(xd - sensor_locate[select[i][0]][0]) + np.square(yd - sensor_locate[select[i][0]][1])) / vg - np.sqrt(np.square(xd - sensor_locate[select[i][1]][0]) + np.square(yd - sensor_locate[select[i][1]][1])) / vg)
    return count

In [82]:
# 返回符合逆伽马分布的随机数
def fuc_ig(fuc_ig_theta, def_ig_tof):
    print("fuc_ig", fuc_ig_theta, def_ig_tof)
    return invgamma.rvs(2.5, scale=(1 / fuc_q(fuc_ig_theta, def_ig_tof)))

In [83]:
# MCMC方法本体
def def_mcmc(def_mcmc_array, def_mcmc_tof):
    print("def_mcmc", def_mcmc_array, def_mcmc_tof)
    def_theta = np.zeros((NT,2))
    def_theta[0] = [250, 250]
    LK = [1, 1]
    fuc_rapid_pdf_pdf = np.zeros(plant_size)
    fuc_rapid_pdf_pdf = fuc_rapid_pdf(def_mcmc_array)
    for i in range(1, NT):
        theta_1 = def_theta[i - 1][0] + 2 * LK[0] * (2 * random.random() - 1)
        theta_2 = def_theta[i - 1][1] + 2 * LK[1] * (2 * random.random() - 1)
        theta_k = [theta_1, def_theta[i - 1][1]]
        alpha = fuc_p(fuc_rapid_pdf_pdf, theta_k, [def_theta[i - 1][0], def_theta[i - 1][1]])
        r = alpha * math.exp(-0.5 * fuc_ig(def_theta[i - 1], def_mcmc_tof) * (fuc_q(theta_k, def_mcmc_tof) - fuc_q([def_theta[i - 1][0], def_theta[i - 1][1]], def_mcmc_tof)))
        R = random.random()
        if R < r:
            def_theta[i][0] = theta_1
            LK[0] = LK[0] * 1.01
        else:
            def_theta[i][0] = def_theta[i - 1][0]
            LK[0] = LK[0] / 1.07
        theta_k = [def_theta[i][0], theta_2]
        alpha = fuc_p(fuc_rapid_pdf_pdf, theta_k, [def_theta[i][0], def_theta[i - 1][1]])
        r = alpha * math.exp(-0.5 * fuc_ig(def_theta[i - 1], def_mcmc_tof) * (fuc_q(theta_k, def_mcmc_tof) - fuc_q([def_theta[i][0], def_theta[i - 1][1]], def_mcmc_tof)))
        R = random.random()
        if R < r:
            def_theta[i][1] = theta_2
            LK[1] = LK[1] * 1.01
        else:
            def_theta[i][1] = def_theta[i - 1][1]
            LK[1] = LK[1] / 1.07
    return def_theta

In [84]:
# 制图函数，但是有点小问题
def def_plot_fig(def_pf_theta, def_pf_title):
    def_pf_fig, (def_pf_ax1, def_pf_ax2, def_pf_ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
    def_pf_x = np.linspace(0, NT-1, NT)
    def_pf_y1 = def_pf_theta[:, 0]
    def_pf_y2 = def_pf_theta[:, 1]
    def_pf_ax1.set_title('MCMC samples')
    def_pf_ax1.plot(def_pf_x, def_pf_y1, label='x-coordinate')
    def_pf_ax1.plot(def_pf_x, def_pf_y2, label='y-coordinate')
    def_pf_ax1.legend()
    def_pf_ax1.set_ylim([0, plant_size[0]])
    def_pf_ax1.set_xlabel('simples')
    def_pf_ax1.set_ylabel('location coordinates(mm)')

    def_pf_x_joint, def_pf_x_bins = np.histogram(def_pf_theta[:, 0], bins = np.arange(0, plant_size[0] + 1, 1))
    def_pf_y_joint, def_pf_y_bins = np.histogram(def_pf_theta[:, 1], bins = np.arange(0, plant_size[1] + 1, 1))
    def_pf_x_joint.flatten()
    def_pf_y_joint.flatten()
    def_pf_pdf_joint = np.outer(def_pf_x_joint, def_pf_y_joint)
    def_pf_ax2_img = def_pf_ax2.imshow(def_pf_pdf_joint, cmap = 'viridis')
    def_pf_ax2.set_title('MCMC joint PDF')
    def_pf_ax2.set_xlabel('x-coordinate')
    def_pf_ax2.set_ylabel('y-coordinate')

    def_pf_prior = np.linspace(0, plant_size[0] + 1, plant_size[0] + 1)
    def_pf_x_prior = stats.norm.pdf(def_pf_prior, def_p_x, scale=def_p_x_scale)
    def_pf_y_prior = stats.norm.pdf(def_pf_prior, def_p_y, scale=def_p_y_scale)
    def_pf_pdf_prior = np.outer(def_pf_x_prior, def_pf_y_prior)
    def_pf_ax3_img = def_pf_ax3.imshow(def_pf_pdf_prior, cmap = 'viridis')
    def_pf_ax3.set_title('MCMC prior PDF')
    def_pf_ax3.set_xlabel('x-coordinate')
    def_pf_ax3.set_ylabel('y-coordinate')

    def_pf_fig.suptitle('MCMC method: ' + def_pf_title)

# 图有问题还没改

In [ ]:
locate = "20180605T150315_D9/pc_f100kHz.h5"
SHM_Data_baseline = def_get_array(SHM_Data_baseline_locate)

tof, dmg_array = def_multi_tof(locate)
theta = def_mcmc(dmg_array, tof)

def_plot_fig(theta, '[180, 345], [10, 10]')